# ***DIPLOMBA***

Для начала скачиваем необходимые пакеты при помощи !pip install

In [15]:
!pip install Counter
!pip install torch
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Пакеты установлены, а значит пора импортировать нужные нам библиотеки.

In [16]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

Как только библиотеки импортированы можно начать готовить данные для нашей Diplomba


В строку TRAIN_TEXT_FILE_PATH = 'сюда нужно указать путь для собранных данных (датасета) в формате txt'


Пример TRAIN_TEXT_FILE_PATH = 'DIPLOMBA.txt'

В данном промежутке мы токенизируем текст или же превращаем буквы, пробелы, знаки препинания и т.д. в индексы присваивая им цифру как пример ',' => 1 
Получается своего рода словарь где каждая символ имеет свой индекс.

In [17]:
TRAIN_TEXT_FILE_PATH = '/content/DATASET_LIST_ONE.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'е', 'н', 'т', 'и', 'р', 'с', 'в', 'м', 'л', 'п', 'к', 'ы', 'я', 'б', 'у', 'д', '.', 'ц', '\t', 'з', ',', 'й', 'г', 'ч', 'ь', 'ф', 'х', 'ж', 'ш', 'О', 'ъ', 'В', '«', '»', '\n', 'ё', '1', 'Н', 'З', 'П', 'Р', 'ю', 'U', 'L', 'M', 'У', 'Е', ':', '2', '-', 'e', 'i', 'И', 'n', 'щ', 't', 'С', 'a', 'o', 'A', 'r', 'd', 'l', 'А', 'э', 'S', 'I', 's', 'm', 'g', 'М', 'b', 'C', '+', 'Л', 'T', 'P', 'y', 'К', 'u', 'D', 'w', 'J', 'p', 'Г', '3', 'Т', 'v', 'N', 'k', 'O', '"', 'Ш', 'c', 'W', 'R', '7', 'Э', 'Б', 'H', 'Ф', 'Ц', 'Ь', '–', 'h', 'Ч', 'G', 'F', 'Д', 'E', 'Q']


Теперь когда у нас имеется собственный словарь нужно сделать из него данные для обучения Diplomba

Будем использовать данные для обучения и ответы/подсказки для нашей сети. Примерно это будет выглядеть так -> 

Данные для обучения = ВК на тму

Наша подсказка = КР на тем 

В итоге все вместе = ВКР на тему.

In [18]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

А сейчас мы видим функцию, которая будет предсказывать текст с помощью обученной Diplomba.

Это позволит нам смотреть, что генерирует сеть пока обучается.

In [19]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

Это то место где Diplomba получает знания. 

In [20]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

Тут начинается обучение. Так же его можно прервать, когда Loss перестанет снижаться или в любой другой момент. 

In [21]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.5553007078170777
 довлет требования проработа соотвует требования работа совтетствует требования проработа работа полно пработа соответствует требования системым по сответветствует труетвет струетвет выполненная работа
Loss: 0.7678862309455872
 сполного проработа полностью соответствует требованиям, предъявляемым к дипломным проектам, а её автор заслуживает присвоения квалификации специалиста.	Оценка «Удовлетворительно».	1. Есть нарушения в 
Loss: 0.42749770522117614
 программной программной системы отображены с нарушением нотации UML, на диаграме отсутствует конкретика по выбору ПО.
 Разработка алгоритма опреданных системы отображены с нарушением нотации UML, на д
Loss: 0.31147494703531264
 предвалификации специалиста.	Оценка «Удовлетворительно».	1. Есть нарушения в оформлении работы, не представленная структура реальной базы данных, на диаграмме развертывания системы компоненты программ
Loss: 0.24440260350704193
 интерфейс.
 Разработка алгоритма филичный инкриков обучения и п

KeyboardInterrupt: ignored

После обучения мы наконец можем посмотреть работу Diplomba

Для этого нужно написать оценку работы и ее тему в поле

start_text=' '

Пример start_text='ВКР на тему генерация игровых миров. Оценка "отлично" ' 

если необходимо получить замечания то нужно дописать это слово в start_text

Размерность prediction_len=250 можно менять

Как и случаемость геренируемого текста temp=0.3

In [ ]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.5, 
    prediction_len=165, 
    start_text=''
    )
)